In [56]:
import pandas as pd
from datetime import datetime

In [ ]:
def payer_category(payer_category):
    cats = {
        'DISCOUNTED CASH PRICE INPATIENT': 'cash', 
        'DISCOUNTED CASH PRICE INPATIENT': 'cash', 
        'Gross Charge': 'gross',
        'DE-IDENTIFIED MIN': 'min',
        'DE-IDENTIFIED MAX': 'max'
    }
    try:
        return cats[payer_category]
    except KeyError:
        return "payer"

In [66]:
file = "621811413_South Baldwin_standardcharges.csv"
ccn = '010083'
url = 'https://www.southbaldwinrmc.com/Uploads/Public/Documents/charge-masters/621811413_South%20Baldwin_standardcharges.csv'

In [58]:
df = pd.read_csv(file)

In [59]:
df = df.rename(columns={
    "SVCCD": 'procedure_code',
    'Description': 'description',
    'CPT': 'hcpcs_cpt',
    'As of Date': 'file_last_updated'
})

In [60]:
df['code'] = df['hcpcs_cpt']
df["file_last_updated"] = pd.to_datetime(df["file_last_updated"])

In [61]:
cols = df.columns.tolist()
id_vars = cols[:cols.index('hcpcs_cpt')+1]
id_vars.extend(['file_last_updated', 'code'])

payer_cat = cols[cols.index('DISCOUNTED CASH PRICE INPATIENT'):]
payer_cat.pop(-1) # Remove code from list
payer_cat.append('Gross Charge')

df = pd.melt(df, id_vars=id_vars, value_vars=payer_cat, var_name='payer_desc', value_name='rate')
payer_cat

['procedure_code', 'description', 'hcpcs_cpt', 'file_last_updated', 'code']


In [62]:
df['patient_class'] = df['payer_desc'].apply(lambda x: 'inpatient' if x.endswith('INPATIENT') else ('outpatient' if x.endswith('OUTPATIENT') else 'na'))

In [64]:
df["plan_name"] = df['payer_desc'].apply(lambda x: x.split(" - ")[-1].strip() if " - " in x else pd.NA)

In [67]:
tin = file.split("_")[0]
tin = tin[:2] + "-" + tin[2:]

df["filename"] = file
df["url"] = url
df["hospital_ccn"] = ccn
df["hospital_tin"] = tin

In [68]:
df

,procedure_code,description,hcpcs_cpt,file_last_updated,code,payer_desc,rate,patient_class,plan_name,filename,url,hospital_ccn,hospital_tin
0,8650033,TC99M PENTETAT-DOSE,A9539,2023-02-13,A9539,DISCOUNTED CASH PRICE INPATIENT,1605.852000,inpatient,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
1,1314655,BORRELIA BURG.CONFIR,86617,2023-02-13,86617,DISCOUNTED CASH PRICE INPATIENT,231.933000,inpatient,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
2,1973235,"THROMBOLY,CORONRY,IV",92977,2023-02-13,92977,DISCOUNTED CASH PRICE INPATIENT,778.902000,inpatient,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
3,1621628,US-PLC LOC DVCE 1ST,MANUL,2023-02-13,MANUL,DISCOUNTED CASH PRICE INPATIENT,535.359000,inpatient,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
4,1211040,GENTAMICIN UPTO 80MG,J1580,2023-02-13,J1580,DISCOUNTED CASH PRICE INPATIENT,5.382000,inpatient,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97879,SURG,Ca screen;flexi sigmoidscope,G0104,2023-02-13,G0104,Gross Charge,11908.875000,na,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
97880,SURG,Colorectal scrn; hi risk ind,G0105,2023-02-13,G0105,Gross Charge,12905.830870,na,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
97881,SURG,Colon ca scrn not hi rsk ind,G0121,2023-02-13,G0121,Gross Charge,12080.911730,na,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
97882,SURG,Inj for sacroiliac jt anesth,G0260,2023-02-13,G0260,Gross Charge,8695.808846,na,NaN,621811413_South Baldwin_standardcharges.csv,https://www.southbaldwinrmc.com/Uploads/Public...,010083,62-1811413
